In [ ]:
# Imports
import dotenv
import os

import imaplib
import email
from email.header import decode_header

In [ ]:
# Load environment variables
dotenv.load_dotenv(dotenv.find_dotenv())
# server_address = os.getenv("IMAP_SERVER_ADDRESS")
imap_server = os.getenv('IMAP_SERVER_ADDRESS')
imap_port = os.getenv('IMAP_SERVER_PORT')
email_address = os.getenv('EMAIL_ADDRESS')
email_password = os.getenv('EMAIL_PASSWORD')

# Connect to the IMAP server
mail = imaplib.IMAP4_SSL(imap_server, imap_port)
mail.login(email_address, email_password)
mail.select('inbox')

In [ ]:
# Search for unseen emails
status, messages = mail.search(None, '(UNSEEN)') 
messages = messages[0].split()

In [ ]:
for mail_id in messages:
    # Fetch the email
    status, data = mail.fetch(mail_id, '(RFC822)')
    raw_email = data[0][1]
    email_message = email.message_from_bytes(raw_email)

    # Check the subject
    subject = decode_header(email_message['Subject'])[0][0]
    if isinstance(subject, bytes):
        subject = subject.decode()
    print("Subject:", subject)

    # Process email content and attachments
    for part in email_message.walk():
        # If it's a plain text or HTML part
        if part.get_content_type() == "text/plain" or part.get_content_type() == "text/html":
            body = part.get_payload(decode=True).decode()
            print("Body:", body)

        # If it's an attachment
        if part.get_content_maintype() == 'multipart' or part.get('Content-Disposition') is not None:
            filename = part.get_filename()
            if filename:
                if 'pdf' in filename.lower():
                    filepath = f'storage_volume/{filename}'
                    with open(filepath, 'wb') as f:
                        f.write(part.get_payload(decode=True))
                    print(f'Downloaded {filename} to {filepath}')

In [ ]:
for mail_id in messages[-10:]:  # Just as an example, let's check the last 10 emails
    status, data = mail.fetch(mail_id, '(RFC822)')
    raw_email = data[0][1]
    email_message = email.message_from_bytes(raw_email)

    # Check each part of the email
    for part in email_message.walk():
        if part.get_content_maintype() == 'multipart' or part.get('Content-Disposition') is not None:
            filename = part.get_filename()
            if filename:
                # Decode the file name if it's in encoded-word format
                decoded_filename = decode_header(filename)[0][0]
                if isinstance(decoded_filename, bytes):
                    decoded_filename = decoded_filename.decode()

                # Save the attachment
                attachment_data = part.get_payload(decode=True)
                filepath = f'storage_volume/{decoded_filename}'
                with open(filepath, 'wb') as f:
                    f.write(attachment_data)
                print(f'Downloaded "{decoded_filename}" to {filepath}')

In [ ]:
# Close the connection
mail.close()
mail.logout()